### Streamlit App
In this section of the hands-on-lab, we will utilize Streamlit with Snowpark's Python client-side Dataframe API to create a visual front-end application for all the squirrelly fans out there to consume the insights from this data journey.

In [ ]:
%%writefile include/streamlit_app.py
import sys, os
sys.path.append(os.getcwd()+'/steps')

import streamlit as st
import numpy as np
import pandas as pd
import altair as alt
import pydeck as pdk
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import snowflake.snowpark as snp

# LOAD DATA ONCE
@st.experimental_singleton
def load_data():
    # establish a session with the Snowflake database.
    from snowpark_connection import snowpark_connect
    session, state_dict = snowpark_connect('./include/state.json')
    data = session.table("squirrels_engineered_clusters").to_pandas()
    data.columns = data.columns.str.strip().str.lower()
    return data

def color_map(data, lat, lon, zoom):
   
    data = data[data.primary_fur_color != 'Unknown'].copy()

    data['r'] = np.where(data.primary_fur_color == 'Gray', 128, (np.where(data.primary_fur_color == 'Black', 0, 210)))
    data['g'] = np.where(data.primary_fur_color == 'Gray', 128, (np.where(data.primary_fur_color == 'Black', 0, 105)))
    data['b'] = np.where(data.primary_fur_color == 'Gray', 128, (np.where(data.primary_fur_color == 'Black', 0, 30)))
    data['a'] = 100

    st.write(pdk.Deck(
     map_style='mapbox://styles/mapbox/light-v9',
     initial_view_state=pdk.ViewState(
         latitude=lat,
         longitude=lon,
         zoom=zoom,
         pitch=30,
     ),
     layers=[
         pdk.Layer(
             'ScatterplotLayer',
             data=data[["lon", "lat","r", "g", "b", "a"]],
             get_position=["lon", "lat"],
             get_color=["r", "g", "b", "a"],
             get_radius=50,
         ),
     ],
 ))


# CALCULATE MIDPOINT FOR GIVEN SET OF DATA
@st.experimental_memo
def mpoint(lat, lon):
    return (np.average(lat), np.average(lon))


# FILTER DATA FOR A SPECIFIC DAY, CACHE
@st.experimental_memo
def filterdata_dt(data, day_selected):
    map_data = (
        data.loc[data["date"] == day_selected]
        if day_selected != "All dates"
        else data
    )
    return map_data

# FILTER DATA FOR A SPECIFIC CLUSTER, CACHE
@st.experimental_memo
def filterdata_cluster(data, cluster_selected):
    map_data = (
        data.loc[data["cluster"] == cluster_selected]
        if cluster_selected != "All clusters"
        else data
    )
    return map_data


# STREAMLIT APP LAYOUT

st.set_page_config(layout="wide")

data = load_data()

# FILTERS
date_selectbox = st.sidebar.selectbox(
    "Date", ["All dates"] + sorted(data["date"].unique())
)

shifts = st.sidebar.markdown("Day-shifts")

shift_am_checkbox = st.sidebar.checkbox("AM", True)
shift_pm_checkbox = st.sidebar.checkbox("PM", True)
checked_shifts = list()
if shift_am_checkbox:
    checked_shifts.append("AM")
if shift_pm_checkbox:
    checked_shifts.append("PM")

  # filter by shift
data = data.loc[data["shift"].isin(checked_shifts)]

# filter by date
data = filterdata_dt(data, date_selectbox)

cluster_selectbox = st.sidebar.selectbox(
    "Cluster", ["All clusters"] + sorted(data["cluster"].unique())
)
# filter by cluster
data = filterdata_cluster(data, cluster_selectbox)


# TABS DEFINITION

tab1, tab2 = st.tabs(["Overview", "Clusters"])

with tab1:

    st.title("Central Park Squirrel Census")

    st.write(
        f'If you’ve ever wondered how many squirrels live in New York City’s Central Park, there’s finally an answer:')

    st.metric(label="Squirrel Population", value=str(len(data["unique_squirrel_id"].unique())))


    st.write('That number comes from the first squirrel census of Manhattan’s largest park, '\
        'conducted by Jamie Allen and more than 300 volunteers who made it their mission to count'\
        'and observe the rodents living in the 843 acres of green space.')

    # st.write(' > It immediately became comical to me. Squirrels are an animal that we interact with ' \
    #    'on a daily basis, they’re disease-carrying, and they’re so common that we don’t even pay attention to them.')

    st.markdown("""---""")


    # LAYING OUT THE TOP SECTION OF THE APP - BAR CHARTS
    row1_1, row1_2 = st.columns([1, 1])

    with row1_1:
        """
        ### Time of squirrels sightings
        """

        date_counts = pd.DataFrame(data.groupby(["date", "shift"]).size()).reset_index()
        date_counts.columns = ["Date", "Shift", "Number of squirrel meetings"]
        date_chart = (
            alt.Chart(date_counts)
            .mark_bar()
            .encode(x=alt.X("Date"), y="Number of squirrel meetings", color="Shift")
            .properties(title="Squirrel meetings by date", width=400, height=400)
        )
        st.altair_chart(date_chart)
        st.write(
        f'Squirrels were seen more in the afternoons - '
        f'{len(data.loc[data["shift"] == "PM"])} times. In the morning shifts researchers '
        f'have met the squirrels {len(data.loc[data["shift"] == "AM"])} times.'
    )


    with row1_2:

        """
        ### Squirrel Population by Age Group
        """
        data["age"].replace("?", "Unknown", inplace=True)
        age_counts = data["age"].value_counts().reset_index()

        age_chart = (
            alt.Chart(age_counts)
            .mark_bar()
            .encode(x="age", y="index")
            .properties(title="Squirrels by age", width=400, height=400)
        )

        st.altair_chart(age_chart)
        st.write(
            "It is difficult to define squirrel age, just by looking at it. "
            "This is the reason of a lot of unknown values."
        )

    # LAYING OUT THE MIDDLE SECTION OF THE APP - SOUNDS
    row2_1, row2_2 = st.columns([1, 1])

    with row2_1:
        """
        ### What are NYC Squirrels Doing?
        """
        activity_data = (
        data[["running", "chasing", "climbing", "eating", "foraging"]].sum().reset_index()
        )
        activity_data.columns = ["Activity", "Count"]
        activity_chart = (
            alt.Chart(activity_data)
            .mark_bar()
            .encode(x=alt.X("Activity:O"), y=alt.Y("Count:Q", sort='x'))
            .properties(height=400, width=400)
        )
        st.altair_chart(activity_chart)

        st.write("Squirrels in Central Park were a hungry bunch - " \
        "the top two activities that they were recorded doing was foraging and eating.")


    with row2_2:
        """
        ### NYC Squirrel Behavior Towards Humans
        """
        behaviour_data = (
        data[["approaches", "indifferent", "runs_from"]].sum().reset_index()
        )
        behaviour_data.columns = ["Behaviour", "Count"]
        behaviour_data = (
            alt.Chart(behaviour_data)
            .mark_bar()
            .encode(x="Behaviour:O", y="Count:Q")
            .properties(height=400, width=400)
        )
        st.altair_chart(behaviour_data)

        st.write("What we do know is that not many squirrels approached humans, more ran from humans  " \
            "and most of them were indifferent from humans.")

    if st.checkbox("Show Raw Data"):
        st.dataframe(data.head(20))


with tab2:
    # cluster area

    row3_1, row3_2, row3_3 = st.columns([1, 1, 1])

    with row3_1:
        """
        has_activity cluster 
        """
        fig = plt.figure(figsize=(10, 8))
        sns.countplot(x='cluster',hue='has_activity', data = data[['has_activity', 'cluster']])
        st.pyplot(fig)


    with row3_2:
        """
        is_stressed cluster 
        """
        fig = plt.figure(figsize=(10, 8))
        sns.countplot(x='cluster',hue='is_stressed', data = data[['is_stressed', 'cluster']])
        st.pyplot(fig)


    with row3_3:
        """
        comfy_with_humans cluster 
        """
        fig = plt.figure(figsize=(10, 8))
        sns.countplot(x='cluster',hue='comfy_with_humans', data = data[['comfy_with_humans', 'cluster']])
        st.pyplot(fig)

    # MAP AREA - SETTING THE ZOOM LOCATIONS 
    central_park = [40.785, 73.968]
    zoom_level = 13
    midpoint = mpoint(data["lat"], data["lon"])

    # LAYING OUT THE MAP

    """
    ### Find your happy squirrels
    """
    color_map(data, midpoint[0], midpoint[1], zoom_level)


- Streamlit requires a local webserver to be spun up. When you enable `Incoming Connections`, Deepnote exposes port 8080 on the internet under an address that you can copy and paste in a browser page. 
- In DeepNote, go to the  `Environment` section, click on the Gear icon, and then copy the tunneling link from the `Incoming Connections` section.

In [ ]:
print(f'Wait a few seconds and then copy the link from the Incoming connection section to open your Streamlit application \n')

!streamlit run --theme.base dark include/streamlit_app.py --server.port 8080 \
                                                          --server.headless true